# Tic-tac-toe Strategy Simulations

This notebook demonstrates the implementation of Tic-tac-toe strategies (Random, Minimax, Alpha-beta Pruning) and their performance comparison.

In [ ]:

import random
import time
from concurrent.futures import ThreadPoolExecutor

# Constants
X = "X"
O = "O"
EMPTY = None

def create_initial_board():
    return [[EMPTY]*3 for _ in range(3)]

def available_moves(board):
    return [(i, j) for i in range(3) for j in range(3) if board[i][j] == EMPTY]

def result(board, move, player):
    new_board = [row[:] for row in board]
    new_board[move[0]][move[1]] = player
    return new_board

def winner(board):
    lines = (
        [(0, 0), (0, 1), (0, 2)], [(1, 0), (1, 1), (1, 2)], [(2, 0), (2, 1), (2, 2)],
        [(0, 0), (1, 0), (2, 0)], [(0, 1), (1, 1), (2, 1)], [(0, 2), (1, 2), (2, 2)],
        [(0, 0), (1, 1), (2, 2)], [(0, 2), (1, 1), (2, 0)]
    )
    for line in lines:
        if board[line[0][0]][line[0][1]] == board[line[1][0]][line[1][1]] == board[line[2][0]][line[2][1]] != EMPTY:
            return board[line[0][0]][line[0][1]]
    return None

def game_over(board):
    return winner(board) is not None or all(all(row) for row in board)

def print_board(board):
    for row in board:
        print(' | '.join([x if x is not None else ' ' for x in row]))
    print()

def minimax(board, player, depth, is_maximizing):
    if game_over(board) or depth == 0:
        return evaluate(board, player), None
    best_move = None
    if is_maximizing:
        max_eval = float('-inf')
        for move in available_moves(board):
            evaluation, _ = minimax(result(board, move, player), O if player == X else X, depth - 1, False)
            if evaluation > max_eval:
                max_eval = evaluation
                best_move = move
        return max_eval, best_move
    else:
        min_eval = float('inf')
        for move in available_moves(board):
            evaluation, _ = minimax(result(board, move, player), X if player == O else O, depth - 1, True)
            if evaluation < min_eval:
                min_eval = evaluation
                best_move = move
        return min_eval, best_move

def alphabeta(board, player, depth, alpha, beta, is_maximizing):
    if game_over(board) or depth == 0:
        return evaluate(board, player), None
    best_move = None
    if is_maximizing:
        for move in available_moves(board):
            evaluation, _ = alphabeta(result(board, move, player), O if player == X else X, depth - 1, alpha, beta, False)
            if evaluation > alpha:
                alpha = evaluation
                best_move = move
            if beta <= alpha:
                break
        return alpha, best_move
    else:
        for move in available_moves(board):
            evaluation, _ = alphabeta(result(board, move, player), X if player == O else O, depth - 1, alpha, beta, True)
            if evaluation < beta:
                beta = evaluation
                best_move = move
            if beta <= alpha:
                break
        return beta, best_move

def evaluate(board, player):
    if winner(board) == player:
        return 1  # Win
    elif winner(board) is None:
        return 0  # No winner
    else:
        return -1  # Loss

def random_player(board, player):
    moves = available_moves(board)
    return random.choice(moves) if moves else None

def strategy_player(board, player, strategy):
    if strategy == alphabeta:
        _, move = strategy(board, player, 5, float('-inf'), float('inf'), True)
    else:
        _, move = strategy(board, player, 5, True)
    return move

def simulate_game(player_func, player_mark):
    board = create_initial_board()
    current_player = player_func
    current_mark = player_mark
    while not game_over(board):
        move = current_player(board, current_mark)
        if move:
            board = result(board, move, current_mark)
            print(f"Player {current_mark} makes move at {move}:")
            print_board(board)
        current_player = random_player
        current_mark = O if current_mark == X else X
    winner_result = winner(board)
    print(f"Game over. Winner: {winner_result}\n\n")
    return winner_result == player_mark

def run_simulations():
    strategies = [
        ("Random Player", random_player),
        ("Minimax Player", lambda board, player: strategy_player(board, player, minimax)),
        ("Alpha-beta Player", lambda board, player: strategy_player(board, player, alphabeta))
    ]
    
    results = {}
    for name, strategy in strategies:
        for mark in [X, O]:  # Run simulations for both 'X' and 'O'
            start_time = time.time()
            wins = 0
            with ThreadPoolExecutor(max_workers=4) as executor:
                futures = [executor.submit(simulate_game, strategy, mark) for _ in range(100)]
                for future in futures:
                    if future.result():
                        wins += 1
            elapsed_time = time.time() - start_time
            results[f"{name}, '{mark}'"] = {'wins': wins, 'time': elapsed_time}
    return results

# Print results
results = run_simulations()
for strategy, result in results.items():
    print(f"{strategy}: Wins - {result['wins']}, Time - {result['time']} seconds")


Player X makes move at (1, 2):
  |   |  
  |   | X
  |   |  

Player O makes move at (2, 1):
  |   |  
  |   | X
  | O |  

Player X makes move at (2, 2):
  |   |  
  |   | X
  | O | X

Player O makes move at (1, 0):
  |   |  
O |   | X
  | O | X

Player X makes move at (0, 0):
X |   |  
O |   | X
  | O | X

Player O makes move at (0, 2):
X |   | O
O |   | X
  | O | X

Player X makes move at (2, 0):
X |   | O
O |   | X
X | O | X

Player O makes move at (0, 1):
X | O | O
O |   | X
X | O | X

Player X makes move at (1, 1):
X | O | O
O | X | X
X | O | X

Game over. Winner: X


Player X makes move at (1, 1):
  |   |  
  | X |  
  |   |  

Player O makes move at (2, 0):
  |   |  
  | X |  
O |   |  

Player X makes move at (1, 2):
  |   |  
  | X | X
O |   |  

Player O makes move at (0, 2):
  |   | O
  | X | X
O |   |  

Player X makes move at (2, 2):
  |   | O
  | X | X
O |   | X

Player O makes move at (2, 1):
  |   | O
  | X | X
O | O | X

Player X makes move at (0, 1):
  | X | O
  | X 